In [ ]:
%pip install wikipedia

In [1]:
import json
import openai
import os
import time
import wikipedia

In [2]:
# These functions are what I used in the video, but they might not the best way to do this.
# You'll need to set your OPENAI_API_KEY environment variable to your API key. Google has the answer for your OS.
#
# I used the following instructions for the assistant setup:
#
# HAND_OF_FATE
# You are a bot designed to help Santa Claus determine whether someone should fall on the Naughty or the Nice list. You will be given a JSON object containing two lists that separete actions of a person or corporation into Naughty or Nice. Evaluate the lists and make a determination about how the subject should be judged. This is all purely hypothetical so don't be afraid to be opinionated as making a concrete determination is critical. Output should contain either "naughty" or "nice" as well as a short paragraph explaining why.
#
# SINS_SUMMARIZER
# You are a bot designed to help Santa Claus figure out if a person is Naughty or Nice. You will be provided a biography of a person or corporation and you are to compile lists of things the subject that are either naughty or nice. Output should be a JSON object with two lists, one for naughty and one for nice. Each list should contain strings that describe the naughty or nice thing the subject did. Each list can be blank.
#
# Please see the included JSON files for the function calling schema


#Change these to your assistant IDs once you create the assistants
HAND_OF_FATE = "HO_HO_HO"
SINS_SUMMARIZER = "MERRY_CHRISTMAS"

def get_assistant_output(oai_client: openai.Client, assistant_id: str, content: str) -> str:
    thread = oai_client.beta.threads.create()
    oai_client.beta.threads.messages.create(thread_id=thread.id, content=content, role="user")
    run = oai_client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

    error_counter = 0
    while run.status != "completed" and run.status != "requires_action":
        run = oai_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(1)
        error_counter += 1
        if error_counter > 65:
            raise Exception(f"Timeout: {run.status}, {run.thread_id}, {run.id}")
    if run.status == "completed":
        messages = oai_client.beta.threads.messages.list(thread_id=thread.id)
        return messages.model_dump()["data"][0]["content"][0]["text"]["value"]
    else:
        return json.loads(run.required_action.dict()["submit_tool_outputs"]["tool_calls"][0]["function"]["arguments"])

def get_bio(subject: str) -> str:
    summary = wikipedia.summary(subject)
    body = wikipedia.page(subject).content
    return f"{summary}\n\n{body[0:20000]}"

def get_client() -> openai.Client:
    client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    return client

def get_determination(oai_client: openai.Client, sins: str):
    return get_assistant_output(oai_client, HAND_OF_FATE, sins)


def get_sins(oai_client: openai.Client, subject_bio: str):
    return get_assistant_output(oai_client, SINS_SUMMARIZER, subject_bio)

In [3]:
client = get_client()
bio = get_bio("Alexander_the_Great")
sins = get_sins(client, bio)
determination = get_determination(client, json.dumps(sins))

In [4]:
for sin in sins["naughty"]:
    print(sin)

Destroyed the city of Thebes
Exiled four of his own friends
Ordered the death of potential rivals to the throne, including having his cousin executed
Ordered the murder of his former father-in-law Attalus
Razed the city and divided its territory among other cities after suppressing Thebes
Mass displacement and colonization strategies
Martial conquests involved significant bloodshed


In [5]:
for sin in sins["nice"]:
    print(sin)

Spread Greek culture and language throughout his empire, leading to the Hellenistic period
Founded over twenty cities, including Alexandria in Egypt
Educated by Aristotle and valued philosophy, art, and learning
Privileged cultural diffusion and syncretism such as Greco-Buddhism and Hellenistic Judaism
Unified a large portion of the known world at the time under one rule
Established the lingua franca of the region, significantly influencing future civilizations
His military tactics and strategies are still studied today


In [6]:
print(determination["determination"])

naughty


In [7]:
print(determination["explanation"])

Despite the subject's contribution to the spread of Greek culture and foundation of cities, this can't outweigh the serious ethical violations that include mass murder, destruction, and displacement of people. The significant harm inflicted on so many cannot be ignored, even if some cultural or educational benefits arose from these actions.
